In [1]:
%pip install pinyin
%pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### `Делаем пинь инь красивым чтоб его можно было переделать в палладий` <br>
То есть записываем без тонов, + транскрипция каждого иероглифа через пробел

In [2]:
# для этого есть библиотека пинь инь
import pandas as pd
import pinyin

In [3]:
df = pd.read_csv('chinese_dataset.csv')

In [4]:
# ставим подходящие параметры
def preprocess_pinyin(data):
    for i in range(len(data)):
        data[i] = pinyin.get(data[i], format="strip", delimiter=" ")
    return data

In [5]:
df_pinyin = df.copy()
df_pinyin['Pinyin'] = preprocess_pinyin(df['Chinese'])

In [6]:
df_pinyin

,Chinese,Pinyin,Wade-Giles,Romatzyh
0,爱,ai,ai⁴,ay
1,爱好,ai hao,ai⁴ hao⁴,ay haw
2,八,ba,pa¹,ba
3,爸爸,ba ba,pa⁴ ba,bah ba
4,爸,ba,pa⁴,bah
...,...,...,...,...
5159,罪,zui,tsui⁴,tzuey
5160,罪恶,zui e,tsui⁴ e⁴,tzuey eh
5161,作,zuo,tso⁴,tzuoh
5162,作废,zuo fei,tso⁴ fei⁴,tzuoh fey


In [7]:
# сохраняем
df_pinyin.to_csv('beautiful_pinyin.csv', index=False)

### `Парсинг` <br>
Нужно запарсить табличку пиньинь - палладий.
Собранная табличка будет сетом правил, через который мы из пиньиня добудем палладий.

In [8]:
import requests
from bs4 import BeautifulSoup

In [9]:
url = "https://cidian.ru/palladius"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')

In [10]:
data = []
for row in table.find_all('tr')[1:]:  # скипаем заголовки
    columns = row.find_all('td')
    
    for td in columns:
        parts = td.text.strip().split(" — ") # на сайте значения пиньинь-палладий внутри <td> разделены "—"
        if len(parts) == 2:
            data.append([parts[0].strip(), parts[1].strip()])

In [20]:
data

[['a', 'а'],
 ['ai', 'ай'],
 ['an', 'ань'],
 ['ang', 'ан'],
 ['ao', 'ао'],
 ['ba', 'ба'],
 ['bai', 'бай'],
 ['ban', 'бань'],
 ['bang', 'бан'],
 ['bao', 'бао'],
 ['bei', 'бэй'],
 ['ben', 'бэнь'],
 ['beng', 'бэн'],
 ['bi', 'би'],
 ['bian', 'бянь'],
 ['biao', 'бяо'],
 ['bie', 'бе'],
 ['bin', 'бинь'],
 ['bing', 'бин'],
 ['bo', 'бо'],
 ['bu', 'бу'],
 ['ca', 'ца'],
 ['cai', 'цай'],
 ['can', 'цань'],
 ['cang', 'цан'],
 ['cao', 'цао'],
 ['ce', 'цэ'],
 ['cei', 'цэй'],
 ['cen', 'цэнь'],
 ['ceng', 'цэн'],
 ['ci', 'цы'],
 ['cong', 'цун'],
 ['cou', 'цоу'],
 ['cu', 'цу'],
 ['cuan', 'цуань'],
 ['cui', 'цуй'],
 ['cun', 'цунь'],
 ['cuo', 'цо'],
 ['cha', 'ча'],
 ['chai', 'чай'],
 ['chan', 'чань'],
 ['chang', 'чан'],
 ['chao', 'чао'],
 ['che', 'чэ'],
 ['chen', 'чэнь'],
 ['cheng', 'чэн'],
 ['chi', 'чи'],
 ['chong', 'чун'],
 ['chou', 'чоу'],
 ['chu', 'чу'],
 ['chua', 'чуа'],
 ['chuai', 'чуай'],
 ['chuan', 'чуань'],
 ['chuang', 'чуан'],
 ['chui', 'чуй'],
 ['chun', 'чунь'],
 ['chuo', 'чо'],
 ['da', 'да'],
 [

In [11]:
# сохраняем
df = pd.DataFrame(data, columns=['Pinyin', 'Palladiy'])
df.to_csv('pinyin_palladiy.csv', index=False)

In [12]:
palladiy_rules = pd.read_csv('pinyin_palladiy.csv')
palladiy_rules

,Pinyin,Palladiy
0,a,а
1,ai,ай
2,an,ань
3,ang,ан
4,ao,ао
...,...,...
428,zhuan,чжуань
429,zhuang,чжуан
430,zhui,чжуй
431,zhun,чжунь


### `Преобразуем` уже preprocessed под нужный формат `пиньинь по собранным правилам`

In [13]:
pinyin = pd.read_csv('beautiful_pinyin.csv')
rules = pd.read_csv('pinyin_palladiy.csv')
pinyin['Palladiy'] = [list() for _ in range(len(pinyin))]

In [14]:
# токенайзер
def token(text):
    text = str(text)
    words = text.split()
    return words

In [15]:
pinyin['Pinyin'] = pinyin['Pinyin'].apply(token)

In [16]:
for row in range(len(pinyin['Pinyin'])):
    for r in range(len(rules['Pinyin'])):
        for word in pinyin['Pinyin'][row]:
            if word == rules['Pinyin'][r]:
                pinyin['Palladiy'][row].append(rules['Palladiy'][r])

In [17]:
def convert_list_to_string(list):
    return ' '.join(list)

In [18]:
pinyin['Pinyin'] = pinyin['Pinyin'].apply(convert_list_to_string)
pinyin['Palladiy'] = pinyin['Palladiy'].apply(convert_list_to_string)
pinyin

,Chinese,Pinyin,Wade-Giles,Romatzyh,Palladiy
0,爱,ai,ai⁴,ay,ай
1,爱好,ai hao,ai⁴ hao⁴,ay haw,ай хао
2,八,ba,pa¹,ba,ба
3,爸爸,ba ba,pa⁴ ba,bah ba,ба ба
4,爸,ba,pa⁴,bah,ба
...,...,...,...,...,...
5159,罪,zui,tsui⁴,tzuey,цзуй
5160,罪恶,zui e,tsui⁴ e⁴,tzuey eh,э цзуй
5161,作,zuo,tso⁴,tzuoh,цзо
5162,作废,zuo fei,tso⁴ fei⁴,tzuoh fey,фэй цзо


In [19]:
# сохраняем
pinyin.to_csv('full_chinese_dataset.csv', index=False)